In [105]:
import numpy as np
import scipy as sci
import scipy.io
from sklearn import svm
from skimage import feature
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import csv
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import Imputer
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
import math
import time
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [106]:
def create_csv(pred, name):
    np.savetxt(name, np.dstack((np.arange(1, len(pred)+1), pred))[0],"%d,%d",header="Id,Category")

In [109]:
#TRANSFORMED DATA
spam_data = scipy.io.loadmat("spam-dataset/spam_data.mat")
spam_train = spam_data['training_data']
spam_labels = spam_data['training_labels'].T
spam_test = spam_data['test_data']
spam_width = spam_train.shape[1]

## Part 1): Code for Decision Trees and Random Forests (plus Testing/Validation/Classifying)

## Decision Tree and Node code

In [71]:
class DecisionTree:
    def __init__(self, height, random = False, m = 25):
        self.height = height
        self.random = random
        self.m = m
        self.root = None
        
    def impurity(self, left_label_hist, right_label_hist):
        # weighted entropy    
        left_size = 0
        for key, val in left_label_hist.items():
            left_size+=val
        right_size = 0
        for key, val in right_label_hist.items():
            right_size+=val
        def calc_entropy(index_set):
            total = 0
            for key, val in index_set.items():
                total+=val
            if total == 0:
                # return big number
                return 9999999999
            probs = {}
            for key, val in index_set.items():
                probs[key] = val/total
            entropy_sum = 0
            for key in index_set:
                if probs[key] == 0:
                    # Case where all the samples go to one class
                    return 0
                entropy_sum += -probs[key] * np.log(probs[key])
            return entropy_sum
        left_entropy = calc_entropy(left_label_hist)
        right_entropy = calc_entropy(right_label_hist)
        weighted_entropy = (left_size * left_entropy + right_size * right_entropy)/(left_size + right_size)
        return weighted_entropy
    
    def segmenter(self, splits, data, labels):
        # for each feature
        feature_entropies = []
        best_rules = []
        rand_features = []
        if self.random:
            index = 0
            feature_map = {}
            while len(rand_features) < self.m:
                i = np.random.randint(len(data.T))
                if not i in rand_features:
                    rand_features.append(i)
                    feature_map[index] = i
                    index+=1
        else:
            feature_map = {i:i for i in range(len(data.T))}
            rand_features = range(len(data.T))
        for feature in rand_features:
            curr_list = []
            counts_dict_0 = {}
            counts_dict_1 = {}
            for i in splits:
                feature_val = data[i][feature]
                label = labels[i]
                curr_list.append(feature_val)
                if label == 0:
                    if feature_val in counts_dict_0:
                        counts_dict_0[feature_val] += 1
                    else:
                        counts_dict_0[feature_val] = 1
                    if not feature_val in counts_dict_1:
                        counts_dict_1[feature_val] = 0
                else:
                    if feature_val in counts_dict_1:
                        counts_dict_1[feature_val] += 1
                    else:
                        counts_dict_1[feature_val] = 1
                    if not feature_val in counts_dict_0:
                        counts_dict_0[feature_val] = 0
            unique_list = list(set(curr_list))
            unique_list.sort()
            
            left_label_hist = {0:0, 1:0}
            right_label_hist = {0:0, 1:0}
            left_label_hist[0]+= counts_dict_0[unique_list[0]]
            left_label_hist[1]+= counts_dict_1[unique_list[0]]
            for i in range(1, len(unique_list)):
                right_label_hist[0]+= counts_dict_0[unique_list[i]]
                right_label_hist[1]+= counts_dict_1[unique_list[i]]

            lowest_entropy = self.impurity(left_label_hist, right_label_hist)
            best_rule = unique_list[0]
            # Testing each split
            for i in range(1, len(unique_list)-1):
                left_label_hist[0]+= counts_dict_0[unique_list[i]]
                left_label_hist[1]+= counts_dict_1[unique_list[i]]
                right_label_hist[0]-= counts_dict_0[unique_list[i]]
                right_label_hist[1]-= counts_dict_1[unique_list[i]]
                curr_entropy = self.impurity(left_label_hist, right_label_hist)
                if curr_entropy < lowest_entropy:
                    lowest_entropy = curr_entropy
                    
                    best_rule = unique_list[i]
            feature_entropies.append(lowest_entropy)
            best_rules.append(best_rule)
            
        # MAKE SURE FEATURE MAPPING IS CORRECT
        best_index = np.argmin(feature_entropies)
        rule = best_rules[best_index]
        best_feature = feature_map[best_index]
        return (best_feature, rule)
    
    def train(self, train_data, train_labels):
        S = []
        for i in range(train_data.shape[0]):
            S.append(i)
        def grow_tree(S, train_data, train_labels, height):

            def get_majority(labels):
                count0 = 0
                count1 = 0
                for i in range(len(new_labels)):
                    if new_labels[i][0] == 0:
                        count0+=1
                    else:
                        count1+=1
                if count0 > count1:
                    return 0
                return 1

            same = True
            val = train_labels[S[0]]
            new_labels = train_labels[S]
            for i in S:
                if val != train_labels[i]:                    
                    same = False
                    break
            if height == 0:
                return Node(None, None, None, get_majority(new_labels))
            if same:
                return Node(None, None, None, int(val[0]))
            else:
                feature, split = self.segmenter(S, train_data, train_labels)
                S_l = []
                S_r = []
                for i in S:
                    if train_data[i][feature] <= split:
                        S_l.append(i)
                    else:
                        S_r.append(i)
                
                if len(S_l) == 0 or len(S_r) == 0:
                    return Node(None, None, None, get_majority(new_labels))
                return Node((feature, split), grow_tree(S_l, train_data, train_labels, height-1), grow_tree(S_r, train_data, train_labels, height-1))
        self.root = grow_tree(S, train_data, train_labels, self.height)
    
    
    def predict(self, test_data):
        predicted_labels = np.zeros((len(test_data), 1))
        for i in range(len(test_data)):
            curr_node = self.root
            while curr_node.label==None:
                feature, rule = curr_node.split_rule
                if test_data[i][feature] <= rule:
                    curr_node = curr_node.left
                else:
                    curr_node = curr_node.right
            predicted_labels[i] = curr_node.label
        return predicted_labels

class Node:
    def __init__(self, split_rule, left, right, label = None):
        self.split_rule = split_rule
        self.left = left
        self.right = right
        self.label = label 
    
    def __str__(self):
        return "[Split rule: " + str(self.split_rule) + " " + str(self.label) + " ]"
    
def benchmark(pred_labels, true_labels):
    errors = pred_labels != true_labels
    err_rate = sum(errors) / float(len(true_labels))
    indices = errors.nonzero()
    return err_rate, indices


## Training and validating for Decision Tree

In [72]:
train, val_set, train_labels, val_labels = train_test_split(spam_train, spam_labels, test_size = int(len(spam_labels)/3))
start = time.time()
classifier_spam = DecisionTree(50)
classifier_spam.train(train, train_labels)
end = time.time()

pred = classifier_spam.predict(val_set)
err_rate, indices = benchmark(pred, val_labels)
print("Validation error rate is:", err_rate)
print("Time elapsed:", end - start)

Validation error rate is: 0.0841067285383
Time elapsed: 39.072932958602905


## Random Forest Code

In [73]:
class RandomForest:
    def __init__(self, num_trees, height, m = 25):
        self.num_trees = num_trees
        self.height = height
        self.m = m
        self.trees = []
    
    def train(self, train_data, train_labels):
        def sample_data(train_data, train_labels):
            sample_list = np.random.randint(len(train_data), size = len(train_data))
            new_train = train_data[sample_list]
            new_labels = train_labels[sample_list]
            new_labels.reshape((len(new_labels), 1))
            return new_train, new_labels
        
        for _ in range(self.num_trees):
            # CHANGED!
            new_train, new_labels = sample_data(train_data, train_labels)
            classifier = DecisionTree(self.height, random = True, m = self.m)
            classifier.train(new_train, new_labels)
            self.trees.append(classifier)
    
    def predict(self, test_data):
        predictions = np.zeros((len(test_data), 1))
        for tree in self.trees:
            pred = tree.predict(test_data)
            predictions+=pred
        predictions/=len(self.trees)
        final_pred = np.where(predictions > 0.5, 1, 0)
        return final_pred

## Training and Validating for random forests (used for Kaggle)

In [110]:
train, val_set, train_labels, val_labels = train_test_split(spam_train, spam_labels, test_size = int(len(spam_labels)/5))
forest_spam = RandomForest(1000, 100, m=40)
start = time.time()
forest_spam.train(train, train_labels)
end = time.time()
pred = forest_spam.predict(val_set)
err_rate, indices = benchmark(pred, val_labels)
print("Validation error rate is:", err_rate)
print("Time elapsed:", end - start)

Validation error rate is: 0.0183752417795
Time elapsed: 5736.431449174881


In [111]:
pred = forest_spam.predict(spam_test)
pred = pred.reshape((len(pred),))

In [112]:
create_csv(pred, "spam.csv")

##Census Data

In [77]:
categorical = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
with open('census_data/train_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    samples_temp = []
    count = False
    for row in reader:
        if not count:
            first_row = row
        if count:
            samples_temp.append(row)
        count = True
        
with open('census_data/train_data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    samples = []
    count = 0
    for row in reader:
        for key, val in row.items():
            if key not in categorical:
                row[key] = int(val)
        samples.append(row)

samples_temp = np.array(samples_temp)
modes = {}

def get_mode(values):
    counts = {}
    for i in values:
        if i in counts:
            counts[i]+=1
        else:
            counts[i]=1
    max_freq = 0
    mode = None
    for key, val in counts.items():
        if val > max_freq:
            max_freq = val
            mode = key
    return mode

i = 0
for feature in samples_temp.T:
    modes[first_row[i]] = get_mode(feature)
    i+=1

for row in samples:
    for key, val in row.items():
        if val == '?':
            row[key] = modes[key]

In [78]:
# Fitting data
v = DictVectorizer(sparse=False)
X = v.fit_transform(samples)
label_index = v.feature_names_.index("label")

In [79]:
# Taking out the label column
census_labels = X.T[label_index].reshape(len(X), 1)
census_data = np.delete(X.T, label_index, 0).T

## Load test data in

In [80]:
with open('census_data/test_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    samples_temp = []
    count = False
    for row in reader:
        if not count:
            first_row = row
        if count:
            samples_temp.append(row)
        count = True

with open('census_data/test_data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    samples = []
    count = 0
    for row in reader:
        for key, val in row.items():
            if key not in categorical:
                row[key] = int(val)
        samples.append(row)

samples_temp = np.array(samples_temp)
modes = {}

def get_mode(values):
    counts = {}
    for i in values:
        if i in counts:
            counts[i]+=1
        else:
            counts[i]=1
    max_freq = 0
    mode = None
    for key, val in counts.items():
        if val > max_freq:
            max_freq = val
            mode = key
    return mode

i = 0
for feature in samples_temp.T:
    modes[first_row[i]] = get_mode(feature)
    i+=1

for row in samples:
    for key, val in row.items():
        if val == '?':
            row[key] = modes[key]

In [81]:
X = v.transform(samples)
label_index = v.feature_names_.index("label")
census_test = np.delete(X.T, 22, 0).T

## Testing and validating decision tree for census 

In [82]:
train, val_set, train_labels, val_labels = train_test_split(census_data, census_labels, test_size = int(len(census_labels)/3))
classifier_census = DecisionTree(50, m = 11)
start = time.time()
classifier_census.train(train, train_labels)
end = time.time()
pred = classifier_census.predict(val_set)
err_rate, indices = benchmark(pred, val_labels)
print("Validation error rate is:", err_rate)
print("Time elapsed:", end - start)

Validation error rate is: 0.184726806014
Time elapsed: 102.51969599723816


## Testing and validating forests for census (Kaggle)

In [113]:
train, val_set, train_labels, val_labels = train_test_split(census_data, census_labels, test_size = int(len(census_labels)/3))
forest_census = RandomForest(500, 200, m = 11)
start = time.time()
forest_census.train(train, train_labels)
end = time.time()
pred = forest_census.predict(val_set)
err_rate, indices = benchmark(pred, val_labels)

In [114]:
print("Validation error rate is:", err_rate)
print("Time elapsed:", end - start)

Validation error rate is: 0.140905757242
Time elapsed: 5301.995204210281


In [115]:
pred = forest_census.predict(census_test)
pred = pred.reshape((len(pred),))

In [116]:
create_csv(pred, "census.csv")

## Part 3 a): Extra features for spam dataset

For the spam dataset, I parsed the spam and ham training data using the counts script that was provided in previous homeworks to see the most frequent words for both ham and spam. Then I used the bag-of-words model and used the top 400 words for each of ham and spam as my features. I also manually added some custom features such as the counts of various punctuation marks.

## Part 3 b): Report of results

The decision tree had a 8.41% error rate on the validation set. The random forest had a 2.44% error rate on the validation set. These error rates are shown in Part (1) of this notebook. My best Kaggle score for the spam dataset was 0.94283, which was obtained using random forests.

## Part 3 c): Splits in decision tree for a chosen data point 

In [96]:
# List of features (copy and pasted from featurize.py output)
# Should make this more easy to produce later, so I can generalize
spam_feature_list = [';', '$', '#', '!', '(', '[', '&', '-', '.', '%', '"', '1', '2', 'the', 'to', 'and', 'of',
 '3', 'a', 'in', 'you', 'for', 'this', 'is', '4', 'your', 'subject', 'with','that', '5', 's', 'be', 'or', 'on',
 '_', 'as', 'are', 'i', 'we', 'it', 'not', 'our', 'com', 'http', 'from', '6', '7', 'have', 'all', 'no', 'at', 'company',
 '0', 'will', 'by', '8', 'e', 'can', 'an', 'more', 'here', 'www', '00', 'any', 'if', '10', '9', 'information', 'font',
 'me', 't', 'only', 'td', 'has', 'get', 'please', 'd', 'statements', 'email', 'us', 'price', 'new', 'may', 'nbsp', 'my',
 'one', 'p', '15', 'do', 'was', 'now', 'up', 'height', 'time', 'its', 'out', 'these', '99', 'o', 'free', 'within',
 '11', 'pills', 'size', 'width', 'but', 'r', 'about', 'over', '2004', 'stock', 'b', 'other', 'message', 'money',
 '12', 'which', 'investment', 'm', 'u', 'c', 'report', 'their', '20', 'been', 'inc', 'just', 'securities', '14',
 'business', 'online', '100', 'click', 'best', 'looking', '60', 'there', 'what', '000', 'mail', 'like', 'contact',
 'computron', 'so', 'prices', 'align', 'future', 're', 'tr', 'x', 'news', 'need', 'products', 'v', 'into', 'net',
 '50', 'save', 'want', 'go', 'also', 'n', 'forward', '13', 'many', 'use', '25', 'microsoft', 'windows', '30', '69',
 'today', 'would', 'color', 'they', 'million', 'international', 'th', 'link', 'software', 'face', '16', 'border',
 'most', 'internet', 'than', 'without', 'market', 'make', 'account', 'available', 'href', 'line', 'could', 'g',
 'professional', 'high', 'info', 'l', 'should', 'such', 'companies', 'see', 'order', 'viagra', 'world', 'office',
 'how', '95', 'through', 'br', '90', '17', 'some', 'before', 'gas', 'act', 'number', '18', 'when', 'xp', '24',
 'reply', 'product', '63', 'stocks', 'who', 'take', 'adobe', 'address', 'special', 'offer', 'src', 'service',
 'site', 'don', 'advice', 'first', 'back', 'he', 'stop', 'nd', 'results', 'remove', 'pt', 'his', 'low',
 'based', 'f', 'section', 'made', 'system', 'cialis', '21', '80', 'know', 'oil', 'security', 'home',
 'visit', 'them', 'shares', '40', 'very', 'center', 'send', 'energy', 'after', 'soft', 'performance', '120',
 '2003', '161', 'sent', 'own', 'list', 'services', 'dollars', 'prescription', 'h', 'year', 'buy', 'day', 'off',
 'meds', 'below', 'newsletter', 'phone', 'html', 'next', 'am', 'y', 'were', 'family', 'two', 'even', 'had',
 'being', 'long', 'style', 'days', 'sales', 'said', 'hours', 'per', 'works', '19', 'people', '45', 'good',
 'full', 'well', '44', 'name', 'quality', 'top', 'cd', 'paliourg', 'receive', 'does', 'she', 'because',
 'investing', 'due', 'fact', 'every', 'find', 'details', '22', 'limited', 'offers', 'voip', 'interest',
 'above', 'de', 'ms', '27', 'w', '32', 'pro', 'same', 'great', '23', 'via', 'life', 'then', 'events', 've',
 'oo', 'technology', 'last', 'index', 'php', 'health', 'under', 'fax', 'much', 'bgcolor', 'sale', 'real',
 'mg', 'look', 'part', 'china', 'read', 'hi', 'those', 'drugs', 'k', 'right', 'group', 'shipping', 'including',
 'include', 'trade', 'less', 'industry', 'strong', 'her', 'registered', 'check', 'way', 'help', 'easy', 'further',
 '2005', 'month', 'pain', 'past', 'provided', 'pay', 'week', 'copy', 'customers', 'work', 'process', '36', 'gif', 
 'risks', 'content', 'thank', 'where', 'watch', 'mobile', 'biz', 'must', 'believe', 'web', 'cs', 'wish',
 'expectations', 'ect', 'hou', 'enron', '2000', 'deal', 'meter', 'cc', 'pm', 'hpl', '2001', 'daren', 'thanks',
 '01', 'corp', 'mmbtu', 'j', 'forwarded', '03', 'farmer', 'let', 'attached', 'xls', '02', '04', 'contract',
 'volume', 'robert', 'sitara', '05', 'nom', '09', '08', 'texas', 'volumes', 'questions', 'pec', 'deals', 'ena',
 'bob', 'flow', 'file', 'change', 'production', '06', 'call', 'following', '07', '31', 'nomination', 'gary',
 'ticket', '713', 'daily', 'mary', 'march', 'april', 'july', 'original', 'plant', 'melissa', '28', 'houston',
 'effective', 'teco', 'vance', 'tenaska', 'fw', 'george', 'june', 'changes', '26', 'ami', 'pat', 'purchase',
 'julie', 'smith', 'set', '29', 'transport', 'jackie', 'agreement', 'north', 'aimee', 'january', 'noms', 'lisa',
 'tap', 'desk', 'america', 'susan', 'david', 'pipeline', 'october', 'actuals', 'hsc', 'michael', 'mark', 'each',
 'cotten', 'needs', 'think', 'august', 'management', 'taylor', 'megan', 'iv', 'steve', 'chokshi', 'friday',
 'request', 'delivery', 'john', 'fyi', 'tom', 'duke', 'hplc', 'point', 'co', 'date', 'september', 'did', 'tu',
 'november', 'wellhead', 'december', 'clynes', 'february', 'eastrans', 'total', 'feb', 'activity', 'spot', 'lloyd',
 'tickets', 'issue', 'counterparty', 'meeting', 'sure', 'graves', 'unify', 'currently', 'fuel', 'brian', 'team',
 'monday', 'll', 'meters', 'aol', 'resources', 'revised', 'na', 'until', 'lee', '98', 'still', 'actual', 'down',
 'utilities', 'pg', 'supply', 'global', 'contracts', 'txu', 'jan', 'tx', 'meyers', 'both', 'howard', 'hplno',
 'rate', 'rita', 'thursday', 'fee', 'give', 'lannou', 'received', 'again', 'created', 'end', 'nominations',
 'hanks', 'between', 'since', 'young', 'james', 'going', 'note', 'allocation', 'trading', 'demand', 'entered']

spam_feature_mapping = {}
for i in range(len(spam_feature_list)):
    spam_feature_mapping[i] = spam_feature_list[i]

In [97]:
def list_splits(tree, data, mapping):
    node = tree.root
    while node.label == None:
        feature, thresh = node.split_rule
        data_val = data[feature]
        if data_val <= thresh:
            print(mapping[feature], "<=", thresh)
            node = node.left
        else:
            print(mapping[feature], ">", thresh)
            node = node.right
    print("Data point classified as", node.label)

In [98]:
list_splits(classifier_spam, spam_test[2], spam_feature_mapping)

enron <= 0.0
! <= 0.0
hpl <= 0.0
thanks > 0.0
http <= 1.0
. > 0.0
and <= 12.0
Data point classified as 0


## Part 3 d): Most common splits at root for random forests

In [89]:
def common_splits(forest, mapping):
    split_rules_counts = {}
    for tree in forest.trees:
        root = tree.root
        split_rule = root.split_rule
        if split_rule not in split_rules_counts:
            split_rules_counts[split_rule] = 1
        else:
            split_rules_counts[split_rule] += 1
    print("Most common splits:")
    for _ in range(5):
        best = max(split_rules_counts, key=lambda i: split_rules_counts[i])
        print(mapping[best[0]], "<=", best[1], "(" + str(split_rules_counts[best]) + " occurences)")
        del split_rules_counts[best]

In [99]:
common_splits(forest_spam, spam_feature_mapping)

Most common splits:
sitara <= 0.0 (11 occurences)
http <= 0.0 (10 occurences)
questions <= 0.0 (9 occurences)
enron <= 0.0 (9 occurences)
pm <= 0.0 (9 occurences)


## Part 4 a): Extra features for census dataset

For the categorical variables, I followed the spec's "Easy" suggestion, and just mapped each category to a binary variable, effectively creating new feature columns for each value in a categorical variable. As for dealing with missing attributes, I employed the "Easy" technique suggested by the spec and simply replaced the missing value with the mode of that feature.

## Part 4 b): Report of results

The decision tree had a 18.47% error rate on the validation set. The random forest had a 14.00% error rate on the validation set. These error rates are shown in Part (1) of this notebook. My best Kaggle score for the census dataset was 0.78427, which was obtained using random forests.

## Part 4 c): Splits in decision tree for a chosen data point 

In [92]:
census_feature_mapping = {}
v.feature_names_
count = 0
for item in v.feature_names_:
    if item != 'label':
        census_feature_mapping[count] = item
    count+=1

In [103]:
list_splits(classifier_census, census_test[2], census_feature_mapping)

marital-status=Married-AF-spouse <= 0.0
capital-gain <= 6849.0
education-num > 12.0
age <= 32.0
hours-per-week <= 44.0
capital-loss <= 2001.0
marital-status=Divorced <= 0.0
capital-loss <= 1504.0
capital-gain > 4101.0
age > 29.0
Data point classified as 0


## Part 4 d): Most common splits at root for random forests

In [104]:
common_splits(forest_census, census_feature_mapping)

Most common splits:
relationship=Other-relative <= 0.0 (12 occurences)
marital-status=Married-AF-spouse <= 0.0 (9 occurences)
education-num <= 12.0 (8 occurences)
race=White <= 0.0 (8 occurences)
marital-status=Married-spouse-absent <= 0.0 (7 occurences)


## Part 5 a)

For stopping criteria, I have a depth parameter which makes it so that a node in the decision tree stops growing once it reaches a certain depth. The only splitting criteria I used was weighted entropy. As for dealing with missing attributes, I employed the "Easy" technique suggested by the spec and simply replaced the missing value with the mode of that feature.

## Part 5 b)

For random forests, I implement bagging for both the samples and the features. I do the techniques suggested in lecture, where for each tree, I create the data matrices by randomly sampling with replacement and creating a size n x d matrix, where n is the number of original samples. For features, I only split on sqrt(d) features where d is the number of features. The decision tree I used for the forests only differed in the bagging techniques used, but were otherwise the same.

# Copy of featurize.py code and count.py code (For reference, not to run)

## featurize.py

In [ ]:
'''
    **************** PLEASE READ ***************
    
    Script that reads in spam and ham messages and converts each training example
    into a feature vector
    
    Code intended for UC Berkeley course CS 189/289A: Machine Learning
    
    Requirements:
    -scipy ('pip install scipy')
    
    To add your own features, create a function that takes in the raw text and
    word frequency dictionary and outputs a int or float. Then add your feature
    in the function 'def generate_feature_vector'
    
    The output of your file will be a .mat file. The data will be accessible using
    the following keys:
    -'training_data'
    -'training_labels'
    -'test_data'
    
    Please direct any bugs to kevintee@berkeley.edu
    '''

from collections import defaultdict
import glob
import re
import scipy.io

NUM_TRAINING_EXAMPLES = 5172
NUM_TEST_EXAMPLES = 5857

BASE_DIR = './'
SPAM_DIR = 'spam/'
HAM_DIR = 'ham/'
TEST_DIR = 'test/'

# ************* Features *************

# Features that look for certain words
def freq_pain_feature(text, freq):
    return float(freq['pain'])

def freq_private_feature(text, freq):
    return float(freq['private'])

def freq_bank_feature(text, freq):
    return float(freq['bank'])

def freq_money_feature(text, freq):
    return float(freq['money'])

def freq_drug_feature(text, freq):
    return float(freq['drug'])

def freq_spam_feature(text, freq):
    return float(freq['spam'])

def freq_prescription_feature(text, freq):
    return float(freq['prescription'])

def freq_creative_feature(text, freq):
    return float(freq['creative'])

def freq_height_feature(text, freq):
    return float(freq['height'])

def freq_featured_feature(text, freq):
    return float(freq['featured'])

def freq_differ_feature(text, freq):
    return float(freq['differ'])

def freq_width_feature(text, freq):
    return float(freq['width'])

def freq_other_feature(text, freq):
    return float(freq['other'])

def freq_energy_feature(text, freq):
    return float(freq['energy'])

def freq_business_feature(text, freq):
    return float(freq['business'])

def freq_message_feature(text, freq):
    return float(freq['message'])

def freq_volumes_feature(text, freq):
    return float(freq['volumes'])

def freq_revision_feature(text, freq):
    return float(freq['revision'])

def freq_path_feature(text, freq):
    return float(freq['path'])

def freq_meter_feature(text, freq):
    return float(freq['meter'])

def freq_memo_feature(text, freq):
    return float(freq['memo'])

def freq_planning_feature(text, freq):
    return float(freq['planning'])

def freq_pleased_feature(text, freq):
    return float(freq['pleased'])

def freq_record_feature(text, freq):
    return float(freq['record'])

def freq_out_feature(text, freq):
    return float(freq['out'])

# Features that look for certain characters
def freq_semicolon_feature(text, freq):
    return text.count(';')

def freq_dollar_feature(text, freq):
    return text.count('$')

def freq_sharp_feature(text, freq):
    return text.count('#')

def freq_exclamation_feature(text, freq):
    return text.count('!')

def freq_para_feature(text, freq):
    return text.count('(')

def freq_bracket_feature(text, freq):
    return text.count('[')

def freq_and_feature(text, freq):
    return text.count('&')

# added here

def freq_subscribe_feature(text, freq):
    return float(freq['subscribe'])

def freq_unsubscribe_feature(text, freq):
    return float(freq['unsubscrime'])

def freq_dash_feature(text, freq):
    return text.count('-')

def freq_dot_feature(text, freq):
    return text.count('.')

def freq_fast_feature(text, freq):
    return text.count('fast')

def freq_trial_feature(text, freq):
    return text.count('trial')

def freq_product_feature(text, freq):
    return text.count('product')

def freq_thank_feature(text, freq):
    return text.count('thank')

def freq_click_feature(text, freq):
    return text.count('click')

def freq_link_feature(text, freq):
    return text.count('link')

def freq_free_feature(text, freq):
    return text.count('free')

def freq_discount_feature(text, freq):
    return text.count('discount')

def freq_sale_feature(text, freq):
    return text.count('sale')

def freq_buy_feature(text, freq):
    return text.count('buy')

def freq_payment_feature(text, freq):
    return text.count('payment')

def freq_saving_feature(text, freq):
    return text.count('saving')

def freq_savings_feature(text, freq):
    return text.count('savings')

#more

def freq_http_feature(text, freq):
    return text.count('http')

def freq_made_feature(text, freq):
    return text.count('made')

def freq_warranty_feature(text, freq):
    return text.count('warranty')

def freq_percent_feature(text, freq):
    return text.count('%')

def freq_you_feature(text, freq):
    return text.count('you')

def freq_heart_feature(text, freq):
    return text.count('heart')

def freq_sale_feature(text, freq):
    return text.count('sale')

def freq_low_feature(text, freq):
    return text.count('low')

def freq_price_feature(text, freq):
    return text.count('price')

def freq_prices_feature(text, freq):
    return text.count('prices')

def freq_www_feature(text, freq):
    return text.count('www')

def freq_quotes_feature(text, freq):
    return text.count('\"')

def generate_all_spam():
    with open('spam/counts') as f:
        content = f.readlines(1)
    return content
# --------- Add your own feature methods ----------
def example_feature(text, freq):
    return int('example' in text)

# Generates a feature vector
def generate_feature_vector(text, freq, list_of_words, list_of_words2):
    feature = []
    feature.append(freq_semicolon_feature(text, freq))
    feature.append(freq_dollar_feature(text, freq))
    feature.append(freq_sharp_feature(text, freq))
    feature.append(freq_exclamation_feature(text, freq))
    feature.append(freq_para_feature(text, freq))
    feature.append(freq_bracket_feature(text, freq))
    feature.append(freq_and_feature(text, freq))
    feature.append(freq_dash_feature(text, freq))
    feature.append(freq_dot_feature(text, freq))
    feature.append(freq_percent_feature(text, freq))
    feature.append(freq_quotes_feature(text, freq))
    used = list(avoid)
    for i in list_of_words[:400]:
        if not i in used:
            feature.append(float(freq[i]))
            used.append(i)
    for i in list_of_words2[:400]:
        if not i in used:
            feature.append(float(freq[i]))
            used.append(i)
    print(used)
    return feature

avoid = [';', '$', '#', '!', '(', '[', '&', '-', '.', '%', '\"']
total_list = []

with open('spam/counts.txt') as f:
    for line in f:
        total_list.append(line.split(' ')[0])
total_list2 = []
with open('ham/counts.txt') as f:
    for line in f:
        total_list2.append(line.split(' ')[0])

# This method generates a design matrix with a list of filenames
# Each file is a single training example
def generate_design_matrix(filenames):
    design_matrix = []
    for filename in filenames:
        with open(filename) as f:
            text = f.read() # Read in text from file
            text = text.replace('\r\n', ' ') # Remove newline character
            words = re.findall(r'\w+', text)
            word_freq = defaultdict(int) # Frequency of all words
            for word in words:
                word_freq[word] += 1
            
            # Create a feature vector
            feature_vector = generate_feature_vector(text, word_freq, total_list, total_list2)
            design_matrix.append(feature_vector)
    return design_matrix

# ************** Script starts here **************
# DO NOT MODIFY ANYTHING BELOW

spam_filenames = glob.glob(BASE_DIR + SPAM_DIR + '*.txt')
spam_design_matrix = generate_design_matrix(spam_filenames)
ham_filenames = glob.glob(BASE_DIR + HAM_DIR + '*.txt')
ham_design_matrix = generate_design_matrix(ham_filenames)
# Important: the test_filenames must be in numerical order as that is the
# order we will be evaluating your classifier
test_filenames = [BASE_DIR + TEST_DIR + str(x) + '.txt' for x in range(NUM_TEST_EXAMPLES)]
test_design_matrix = generate_design_matrix(test_filenames)

X = spam_design_matrix + ham_design_matrix
Y = [1]*len(spam_design_matrix) + [0]*len(ham_design_matrix)

file_dict = {}
file_dict['training_data'] = X
file_dict['training_labels'] = Y
file_dict['test_data'] = test_design_matrix
scipy.io.savemat('spam_data.mat', file_dict)

## count.py

In [ ]:
from collections import defaultdict
import glob
import re

WORD_COUNTS = {}
def count(text, freq):
    for word, count in freq.items():
        word = word.lower()
        if word not in WORD_COUNTS:
            WORD_COUNTS[word] = 0
        WORD_COUNTS[word] += count

filenames = glob.glob('*.txt')
for filename in filenames:
    with open(filename) as f:
        text = f.read() # Read in text from file
        text = text.replace('\r\n', ' ') # Remove newline character
        words = re.findall(r'\w+', text)
        freq = defaultdict(int) # Frequency of all words
        for word in words:
            freq[word] += 1
        count(text, freq)

WORDS = sorted(WORD_COUNTS, reverse=True, key=lambda x: WORD_COUNTS[x])

with open('counts.txt', 'w') as f:
    for word in WORDS:
        f.write('{} {}\n'.format(word, WORD_COUNTS[word]))
